# Index data request using _Yahoo! Finance_
In this small tutorial we will create a connector to query the OHLC data of an index and its components. I will be using the SP500 as the reference index and in later steps we will extend it to accept other indices. 

* In **station 1 - raw materials** I will query the data and shape it in a format that we can more easily work with. 

* In **station 2 - processing** I will run some transformations on the data to obtain interesting features, such as the weighed sector returns.

* Additionally, all will be packaged into a class so that we can reutilize it for other projects in **station 5 - distribution**, although the goal is not to productivize it per se. Some changes in the functions done during stations 1 and 2 might be required to generalize the code. 



In [9]:
!pip install yfinance

In [0]:
# IMPORT THE REQUIRED PACKAGES 

import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override() 
from functools import partial, reduce 

## Station 1 - Raw materials 

### Query the components

The first step is to query the components of the index. I will use the list in [Wikipedia](https://en.wikipedia.org/wiki/List_of_S%26P_500_companies) as it is frequently updated. 

In [11]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
tables = pd.read_html(url)
components = tables[0]
components.head(10)

,Symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
5,ATVI,Activision Blizzard,reports,Communication Services,Interactive Home Entertainment,"Santa Monica, California",2015-08-31,718877,2008
6,ADBE,Adobe Inc.,reports,Information Technology,Application Software,"San Jose, California",1997-05-05,796343,1982
7,AMD,Advanced Micro Devices Inc,reports,Information Technology,Semiconductors,"Santa Clara, California",2017-03-20,2488,1969
8,AAP,Advance Auto Parts,reports,Consumer Discretionary,Automotive Retail,"Raleigh, North Carolina",2015-07-09,1158449,1932
9,AES,AES Corp,reports,Utilities,Independent Power Producers & Energy Traders,"Arlington, Virginia",1998-10-02,874761,1981


### Get OHLC data 

Now that we have queried all the components, we will request the symbol information from _Yahoo! Finance_. Fortunately, it is straightforward for a single symbol or a list. 




In [12]:
ticker_data = pdr.get_data_yahoo(list(components['Symbol'].values))


[*********************100%***********************]  505 of 505 completed

2 Failed downloads:
- BF.B: 1d data not available for startTime=-2208988800 and endTime=1587638303. Only 100 years worth of day granularity data are allowed to be fetched per request.
- BRK.B: No data found, symbol may be delisted


## Station 2 - Processing

The requested data is returned in wide format, that means, a big number of columns (~3000). Even though for some analysis it might be useful, some libraries such as [zipline](https://www.zipline.io/) which is a backtesting library developed for python requires a long format to create the data bundles. Another good reason to use the long format is to avoid having column names that are a combination of ticket + variable (e.g. _AAPL_Close_)

That means that for each ticker and date it expects the OHLC values. An advantage of the long format vs the wide is that we can remove empty values from our dataset. This is done using the melt function from pandas

In [13]:
ticker_data_long = ticker_data.reset_index().melt(id_vars = 'Date').dropna().rename(columns = {'variable_1':'Symbol'})
ticker_data_long.head(5)

,Date,variable_0,Symbol,value
9550,1999-11-18,Adj Close,A,27.068665
9551,1999-11-19,Adj Close,A,24.838577
9552,1999-11-22,Adj Close,A,27.068665
9553,1999-11-23,Adj Close,A,24.607880
9554,1999-11-24,Adj Close,A,25.261524


Now we can use both the date and symbol as index for the dataframe and transform the other two columns to have on each column the 

In [14]:
ticker_data_long = ticker_data_long.pivot_table(index = ['Date', 'Symbol'], columns = 'variable_0')
ticker_data_long.head(5)

value             ...                    
variable_0        Adj Close      Close  ...      Open    Volume
Date       Symbol                       ...                    
1962-01-02 BA      0.145748   0.823045  ...  0.837449  352200.0
           CAT     0.135711   1.604167  ...  1.604167  163200.0
           CVX     0.051993   3.296131  ...  0.000000  105600.0
           DIS     0.035517   0.092908  ...  0.092908  817400.0
           DTE     0.506943  30.375000  ...  0.000000    1600.0

[5 rows x 6 columns]

A small cleanup to format the column names..

In [15]:
print(ticker_data_long.columns)
ticker_data_long.columns = [i[1] for i in ticker_data_long.columns]
ticker_data_long.head(5)

MultiIndex([('value', 'Adj Close'),
            ('value',     'Close'),
            ('value',      'High'),
            ('value',       'Low'),
            ('value',      'Open'),
            ('value',    'Volume')],
           names=[None, 'variable_0'])


Adj Close      Close  ...      Open    Volume
Date       Symbol                        ...                    
1962-01-02 BA       0.145748   0.823045  ...  0.837449  352200.0
           CAT      0.135711   1.604167  ...  1.604167  163200.0
           CVX      0.051993   3.296131  ...  0.000000  105600.0
           DIS      0.035517   0.092908  ...  0.092908  817400.0
           DTE      0.506943  30.375000  ...  0.000000    1600.0

[5 rows x 6 columns]

## Station 5 - Distribution

And now is time to wrap it up in a class. For future flexibility I will create an abstract class with the blueprint that all the connectors must folow. 

Then, i will create a base connector `index_gatherer` from which the different index connectors will inherit. This will allow us to extend it to other indices without the hassle of copying and pasting methods from one to another connector. 

In case you are curious about inheritance or abstract classes in python, please refer to [1](https://en.wikipedia.org/wiki/Inheritance_(object-oriented_programming)) and [2](https://www.python-course.eu/python3_abstract_classes.php).


In [0]:
# IMPORT PACKAGES 
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override() 
from functools import partial, reduce 
from abc import ABC, abstractmethod
 
# DEFINE ABSTRACT CLASS
class gatherer(ABC):
  @abstractmethod
  def get_tickers_info(self):
    pass

  @abstractmethod
  def query_tickers(self):
    pass

class index_gatherer(gatherer):
  """
  This is the abstract class for our connectors. 
  They must be created with at least one methods:

  query_tickers: data requesting from Yahoo! Finance or any other source.
  """
  def __init__(self, components_url):
    self.components_url = components_url 
    self.index_info_table = None
    self.components_list = None
    self.ticker_data = None
    super().__init__()
    
  def get_tickers_info(self, tbl_index = 0, ticker_column = 'Symbol'):
    """
    This method requests the components of the index from a url in the form of
    HTML tables and extracts the desired index. 

    It stores in the components_list attribute a python list with all the symbols
    that are the current constituents of the index 
    """
    tables = pd.read_html(self.components_url)
    self.index_info_table = tables[tbl_index]
    self.components_list = self.index_info_table[ticker_column].values

  def query_tickers(self, output_format, data_src):
    """
    This method is extended to request the tickers from Yahoo! Finance. 
    Other sources could also be easily integrated

    Parameters
    ----------
    output_format: `str` with either 'long'(Rc) or 'wide'(rC) output format 
    data_src: `str` source to query from. One of ['yahoo','quandl']
    """ 
    assert data_src in ['yahoo', 'quandl']

    if data_src == 'yahoo':
      self.ticker_data = pdr.get_data_yahoo(list(self.components_list))
    
    if output_format == 'long':
      self.ticker_data = self.ticker_data.reset_index().melt(id_vars = 'Date').dropna().rename(columns = {'variable_1':'Symbol'})
      self.ticker_data = self.ticker_data.pivot_table(index = ['Date', 'Symbol'], columns = 'variable_0')
      self.ticker_data.columns = [i[1] for i in self.ticker_data.columns]


Now we can create an object of class `index_gatherer` and query the available SP500 components OHLC data. 

In [32]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
conn_SP500 = index_gatherer(url)
conn_SP500.get_tickers_info(tbl_index = 0, ticker_column = 'Symbol')
conn_SP500.query_tickers(output_format = 'long', data_src = 'yahoo')
conn_SP500.ticker_data.head(5)

[*********************100%***********************]  505 of 505 completed

2 Failed downloads:
- BF.B: 1d data not available for startTime=-2208988800 and endTime=1587645914. Only 100 years worth of day granularity data are allowed to be fetched per request.
- BRK.B: No data found, symbol may be delisted


Adj Close      Close  ...      Open    Volume
Date       Symbol                        ...                    
1962-01-02 BA       0.145748   0.823045  ...  0.837449  352200.0
           CAT      0.135711   1.604167  ...  1.604167  163200.0
           CVX      0.051993   3.296131  ...  0.000000  105600.0
           DIS      0.035517   0.092908  ...  0.092908  817400.0
           DTE      0.506943  30.375000  ...  0.000000    1600.0

[5 rows x 6 columns]

## Next steps 

We could wrap-up here, but if you remember, we obtained more than just the symbol list from _Wikipedia_, so let's work on that a little bit. 

We will first create an aggregation pipeline to compute the average daily returns by sector and then create a new class `SP500_gatherer` inheriting from `index_gatherer` 


### Station 2 (bonus) - weighed returns by sector

We will extend the base class index_gatherer to be able to compute a weighed indicator based on an informative criterion. For the SP500 we will create a GICS Sector indicator with the daily returns using an equally weighed and also a dollar volume mean. 

First we will create the weighting functions and then the pipeline to apply it group by group. In the case of the SP500, the GICS Sector. 

GICS Sectors provide a classification of companies according to their activities. Please visit [msci](https://www.msci.com/gics) for further information regarding GICS codes.


In [18]:
# Add the GICS Sector information 
# Let's work on daily returns as a %change
variable = 'GICS Sector'
df_grouped = conn_SP500.ticker_data.sort_index(level =[1,0]).pct_change() # sort by Symbol, then Date
df_grouped = df_grouped.reset_index().merge(conn_SP500.index_info_table.loc[:,['Symbol',variable]], on = 'Symbol', how = 'left')
df_grouped.head(5)

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume,GICS Sector
0,1999-11-18,A,NaN,NaN,NaN,NaN,NaN,NaN,Health Care
1,1999-11-19,A,-0.082386,-0.082386,-0.140000,-0.004687,-0.056319,-0.756435,Health Care
2,1999-11-22,A,0.089783,0.089783,0.023256,0.006279,-0.037846,-0.568219,Health Care
3,1999-11-23,A,-0.090909,-0.090909,-0.008523,-0.001560,0.028744,-0.091550,Health Care
4,1999-11-24,A,0.026562,0.026563,-0.038682,0.000000,-0.055882,-0.189504,Health Care


It is now time to test the first weighter, a simple average. As you can see.

In [19]:
import numpy as np 
def weighter(x): return np.nanmean(x)
df_grouped.drop(labels = 'Symbol', axis = 1).groupby(['Date',variable]).agg(lambda x: weighter(x))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: Mean of empty slice
  


Adj Close     Close  ...      Open    Volume
Date       GICS Sector                                  ...                    
1962-01-02 Communication Services  -0.999778 -0.999420  ... -0.999405 -0.790491
           Consumer Staples        -0.999823 -0.981195  ... -0.998647 -0.795753
           Energy                  -0.999492 -0.963982  ... -1.000000 -0.903959
           Health Care             -1.000000 -0.998601  ... -1.000000 -1.000000
           Industrials             -0.996888 -0.961700  ... -0.962090 -0.043828
...                                      ...       ...  ...       ...       ...
2020-04-22 Industrials              0.011212  0.011212  ...  0.018015 -0.005699
           Information Technology   0.039533  0.039533  ... -0.001090 -0.099310
           Materials                0.016308  0.016308  ...  0.024179 -0.088459
           Real Estate              0.013047  0.013047  ...  0.027084 -0.101418
           Utilities                0.026968  0.026968  ...  0.020777 -0.112922

[154884 rows x 6 columns]

Although for the simple average it is an straightforward computation, we should take into consideration that the process was done in three different steps.  

* Compute the aggregation weights: in this case its $1/n_{components}$
* Transform data: here a simple $P_t - P_{t-1} / P_{t-1}$
* Aggregate the results: a simple mean.

We will integrate all three in a wrapper function named `pipeline` that computes the weights, transforms the data and returns an aggregating operation on them. 

So, let's create this three functions:

`transformer_pct`: get the %change for each ticker </br>
`compute_weights_ew`: compute the weights as $\frac{1}{n_{tickers|day=D}}$</br> 
`aggregate_ew`: weighed sum of all the tickers for an industry on a day.


In [0]:
import numpy as np 
def transformer_pct(df):
  """
  This function transforms all numeric columns to intraday %change
   
  parameters
  ----------
  df: `pandas.DataFrame`
  """
  for i in df.columns:
    if np.issubdtype(df[i].dtype, np.number):
      df[i] = df[i].pct_change()
  return df

def compute_weights_ew(df, grouping_vars):
  """
  This function creates the weights to compute a simple average
   
  parameters
  ----------
  df: `pandas.DataFrame`
  """  
  df['weight'] = 1
  df = df.loc[:,grouping_vars+['weight']].groupby(grouping_vars).sum().reset_index().merge(df, on = grouping_vars, suffixes = ['_sum','_ind'])
  df['weight']=df['weight_ind']/df['weight_sum']
  df = df.loc[:,grouping_vars + ['weight', 'Symbol']]
  return df

def aggregate_ew(df, weights, grouping_vars):
  # Set all non-numeric columns as index
  cols_df = [i for i in df.columns.tolist() if not np.issubdtype(df[i].dtype, np.number)]
  cols_wgt = [i for i in weights.columns.tolist() if not np.issubdtype(weights[i].dtype, np.number)]
  common = list(set(cols_df).intersection(set(cols_wgt)))

  df = df.set_index(common)
  weights = weights.set_index(common)

  duplicates = list(set(df.columns).intersection(set(weights.columns)))
  if len(duplicates)>0:
    df = df.drop(duplicates, axis = 1)

  # merge df info and weights
  df = df.merge(weights, right_index = True, left_index = True)
  
  scale = df['weight']

  # apply aggregation
  df = df.apply(lambda x: x*scale)
  cols= df.columns.tolist()
  df = df.reset_index().loc[:,cols+grouping_vars].groupby(grouping_vars).sum()
  return df


In [21]:
variable = 'GICS Sector'
df_ops = conn_SP500.ticker_data.sort_index(level =[1,0]).reset_index()
df_ops = df_ops.merge(conn_SP500.index_info_table.loc[:,['Symbol',variable]], on = 'Symbol', how = 'left')
wgts = compute_weights_ew(df_ops, ['GICS Sector', 'Date'])
agg = aggregate_ew(transformer_pct(df_ops), wgts, ['GICS Sector', 'Date'])
agg.head()

Adj Close     Close  ...    Volume  weight
GICS Sector            Date                             ...                  
Communication Services 1962-01-02  -0.999778 -0.999420  ... -0.790491     1.0
                       1962-01-03   0.013422  0.013423  ... -0.047590     1.0
                       1962-01-04   0.000000  0.000000  ...  0.200000     1.0
                       1962-01-05   0.003312  0.003311  ...  0.000000     1.0
                       1962-01-08  -0.003301 -0.003300  ...  0.333333     1.0

[5 rows x 7 columns]

In [22]:
wgts.groupby(['GICS Sector', 'Date']).sum().describe()

,weight
count,1.548840e+05
mean,1.000000e+00
std,5.946347e-16
min,1.000000e+00
25%,1.000000e+00
50%,1.000000e+00
75%,1.000000e+00
max,1.000000e+00


Now for the second aggregation strategy, by _DollarVolume_ we must modify the `compute_weights` function 

In [0]:
def compute_weights_dvw(df, grouping_vars):
  """
  This function computes the DollarVolume weights of a set of stocks.
   
  parameters
  ----------
  df: `pandas.DataFrame`
  grouping_vars: `list` of `str` with the grouping variables (e.g. date and GICS Sector)
  """  

  df['weight'] = df['Volume']*df['Adj Close']
  df = df.loc[:,grouping_vars+['weight']].groupby(grouping_vars).sum().reset_index().merge(df, on = grouping_vars, suffixes = ['_sum','_ind'])
  df['weight']=df['weight_ind']/df['weight_sum']
  df = df.loc[:,grouping_vars + ['weight', 'Symbol']]
  return df

In [24]:
wgts_dv = compute_weights_dvw(df_ops, ['GICS Sector', 'Date'])
agg_dv = aggregate_ew(transformer_pct(df_ops), wgts_dv, ['GICS Sector', 'Date'])
agg_dv = agg_dv.loc[agg_dv.weight >0]
agg_dv.head()

Adj Close      Close  ...    Volume  weight
GICS Sector            Date                              ...                  
Communication Services 1962-01-02 -24.648604 -24.640132  ... -3.756490     1.0
                       1962-01-03  -1.013425  -1.013431  ... -0.939797     1.0
                       1962-01-08  -1.996699  -1.996700  ...       inf     1.0
                       1962-01-09  -7.019024  -7.019796  ... -2.500000     1.0
                       1962-01-10  -0.509784  -0.509738  ... -1.124920     1.0

[5 rows x 7 columns]

In [0]:
comparison = agg.loc[:,['Adj Close']].merge(agg_dv.loc[:,['Adj Close']], right_index = True, left_index = True, suffixes = ['_ew', '_dvw']).reset_index()
comparison = comparison.loc[comparison['GICS Sector']=='Communication Services',:].set_index('Date')


In [26]:
agg_dv.describe()

,Adj Close,Close,High,Low,Open,Volume,weight
count,1.211970e+05,1.210300e+05,1.198150e+05,1.196830e+05,1.295850e+05,1.494540e+05,1.494880e+05
mean,NaN,NaN,NaN,NaN,NaN,NaN,1.247718e+04
std,NaN,NaN,NaN,NaN,NaN,NaN,2.044182e+06
min,-inf,-inf,-inf,-inf,-inf,-inf,1.660029e-02
25%,-1.043479e+01,-1.048942e+01,-9.601419e+00,-1.014366e+01,-4.646561e+00,-1.035052e+01,5.192742e-01
50%,-6.624419e-01,-6.644940e-01,-6.930288e-01,-7.484860e-01,-2.840522e-01,-2.443832e+00,1.000000e+00
75%,1.072170e+01,1.061031e+01,1.259224e+01,8.996775e+00,3.010528e+00,2.165792e+00,2.210481e+00
max,inf,inf,inf,inf,inf,inf,5.107858e+08


### Station 5 (bonus) - add methods to a class

We will now extend our class SP500 with the aggregation pipeline to compute average return by sector.

In [0]:
class SP500_gatherer(index_gatherer):
  """
  This class is used as a connector to obtain data for the SP500
  """
  def __init__(self, components_url):
        index_gatherer.__init__(self, components_url)
        self.sector_information = None

  def ew_pipeline(self, date_var, grouping_vars):
    df_data = self.ticker_data.sort_index(level =[1,0]).reset_index().merge(self.index_info_table.loc[:,grouping_vars+['Symbol']], on = 'Symbol', how = 'left')
    wgts = compute_weights_ew(df_data, grouping_vars+['Date'])
    self.sector_information = aggregate_ew(transformer_pct(df_data), wgts, grouping_vars+[date_var])


In [28]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
conn_SP500 = SP500_gatherer(url)
conn_SP500.get_tickers_info(tbl_index = 0, ticker_column = 'Symbol')
conn_SP500.query_tickers(output_format = 'long', data_src = 'yahoo')
conn_SP500.ew_pipeline('Date', ['GICS Sector'])

[*********************100%***********************]  505 of 505 completed

2 Failed downloads:
- BF.B: 1d data not available for startTime=-2208988800 and endTime=1587639085. Only 100 years worth of day granularity data are allowed to be fetched per request.
- BRK.B: No data found, symbol may be delisted


In [30]:
conn_SP500.sector_information

Adj Close     Close  ...    Volume    weight
GICS Sector            Date                             ...                    
Communication Services 1962-01-02  -0.999778 -0.999420  ... -0.790491  1.000000
                       1962-01-03   0.013422  0.013423  ... -0.047590  1.000000
                       1962-01-04   0.000000  0.000000  ...  0.200000  1.000000
                       1962-01-05   0.003312  0.003311  ...  0.000000  1.000000
                       1962-01-08  -0.003301 -0.003300  ...  0.333333  1.000000
...                                      ...       ...  ...       ...       ...
Utilities              2020-04-16   0.001759  0.001759  ...  0.062330  0.035714
                       2020-04-17   0.033149  0.033149  ... -0.003989  0.035714
                       2020-04-20  -0.040973 -0.040973  ... -0.005128  0.035714
                       2020-04-21  -0.016277 -0.016277  ...  0.104250  0.035714
                       2020-04-22   0.026968  0.026968  ... -0.112922  0.035714

[154884 rows x 7 columns]

## Recap

In this tutorial we have learned:
* How to create an abstract class connector and define a base index gatherer.
* How to query all the components of the SP500.
* How to extend our base connector with other aggregation methods to compute interesting signals. 


You can find the code for the class and this notebook on [github](https://github.com/franespiga/traiding.git)